In [18]:
from TimeTagger import FileReader
import numpy as np
import easygui

We know that H and V got swapped in BOB so I must correct for it in code. 

In [19]:
#GLOBAL DEFINITIONS
H = 1
V = 2
A = 3
D = 4


In [20]:
#folder = r'C:\Data\\'
folder = "./8thJuly"
file = '2.97mW_1.1.ttbin'
filepath = folder + '/' + file

#filepath = easygui.fileopenbox(default = folder)


originalResolution = 8e-12
finalResolution = 128e-12
maxTime = 0.01



In [21]:
def loadSwabianTimeStamps(fileIn, t = np.inf):
    
        file_reader = FileReader(fileIn)
        stamps = []
        channel = []
        
        
        while file_reader.hasData():
            #n_events = 100000; # Number of events to read at once
            n_events = 100000; # Number of events to read at once
            
            # Read at most n_events.
            data = file_reader.getData(n_events);  # data is an instance of TimeTagStreamBuffer

            channelTmp = data.getChannels()
            stampsTmp = data.getTimestamps()
          #  overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
          #  missed_events = data.getMissedEvents()
            
            for i in range(len(channelTmp)):
                channel.append(channelTmp[i])
                stamps.append(stampsTmp[i])
            #break
            if stamps[-1] - stamps[0] > t:
                break
        return stamps, channel


In [22]:
stamps, channel = loadSwabianTimeStamps(filepath)

In [23]:
#seperate out alice's and bob's events
astamps = []
achannel = []
bstamps = []
bchannel = []
for (s,c) in zip(stamps, channel):
    if (c <=4):
        astamps.append(s)
        achannel.append(c)
    else:
        bstamps.append(s)
        nc = c-4
        if (nc==1):
            nc = 2
        elif (nc == 2):
            nc = 1
        bchannel.append(nc)

In [24]:
len(astamps)

34342903

In [25]:
len(bstamps)

43222014

In [26]:
577447917

577447917

In [27]:
len(bstamps)+len(astamps)

77564917

In [8]:
#convert the resolution to 1/8 of a nanosecond
saverawevents

125.0

In [33]:
astamps[-1]

11057527211237

In [32]:
achannel[-1]

4

In [31]:
len(astamps)

34342903

In [36]:
2197945792/64

34342903.0

In [37]:
def coincount(at,ad,bt,bd,amd=H,bmd=H,cwindow=1000,shift=0):
    #print("coincount: ",amd,bmd,cwindow,shift)
    ia = 0
    ib = 0
    coin_count = 0
    while (ia<len(at) and ib <len(bt)):
        #ignore beacon events
        if (ad[ia]!=amd):
            ia+=1
            continue
        if (bd[ib]!=bmd):
            ib+=1
            continue
            
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            coin_count +=1
            
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return coin_count


In [38]:
#x = coincount(astamps, achannel,bstamps, bchannel, H, H,cwindow=1000,shift=0)
#x

In [39]:
def g2_heisto(at,ad,bt,bd,amd=H,bmd=H,cwindow=1000,search_steps=10,search_direction=1):
    heisto = []
    i = 0
    count = 0
    while(i<search_steps):
        shift = i*cwindow*search_direction
        count = coincount(at, ad,bt, bd, amd,bmd,cwindow=cwindow,shift=shift)
        #print (count)
        heisto.append(count)
        i += 1
    return heisto 

In [40]:
#h = g2_heisto(astamps, achannel,bstamps, bchannel, amd=H, bmd=H,cwindow=500,search_steps=30,search_direction=-1)
#print (h)
#h.index(max(h))

In [41]:
34342903 *64

2197945792

In [42]:
def correct_timestamp(stamps,detectors,sH=0,sV=0,sA=0,sD=0):
    cstamp=[]
    ts = 0
    for (s,d) in zip (stamps,detectors):
        ts = s
        if(d==H): 
            ts+=sH
        elif(d==V):
            ts+=sV
        elif(d==A):
            ts+=sA
        elif(d==D):
            ts+=sD      
        cstamp.append(ts)
    return cstamp

In [43]:
#correct alice HV delay missmatch add 500ps to alice's V 
astamps_corrected = correct_timestamp(astamps,achannel,sH=0,sV=1*500,sA=2*500+5*500,sD=5*500)

In [44]:
#h = g2_heisto(astamps_corrected, achannel,bstamps, bchannel, amd=H, bmd=H,cwindow=500,search_steps=30,search_direction=-1)
#print (h)
#h.index(max(h))

In [45]:
#correct Bob
# add 7 To Bob's A basis
bstamps_corrected = correct_timestamp(bstamps,bchannel,sH=0,sV=0,sA=7*500,sD=0)

In [46]:
#h = g2_heisto(astamps_corrected, achannel,bstamps_corrected, bchannel, amd=H, bmd=H,cwindow=500,search_steps=30,search_direction=-1)
#print (h)
#h.index(max(h))

In [47]:
def match_coincidence(at,ad,bt,bd,cwindow,shift):
    ia = 0
    ib = 0
    acoin_indexes = []
    bcoin_indexes = []
    while (ia<len(at) and ib <len(bt)):
        
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            #coin_count +=1
            acoin_indexes.append(ia)
            bcoin_indexes.append(ib)
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return acoin_indexes,bcoin_indexes
    

In [48]:
ac_indexes,bc_indexes = match_coincidence(astamps_corrected,achannel,bstamps_corrected,bchannel,500,-9*500)

In [57]:
4119059 / len(ac_indexes)

1.2338655444312543

In [49]:
ac_indexes[:10]

[51, 91, 122, 127, 133, 146, 157, 172, 202, 206]

In [50]:
bc_indexes[:10]

[76, 133, 169, 185, 195, 214, 221, 248, 297, 303]

In [53]:
print(astamps_corrected[51],bstamps_corrected[76], bstamps_corrected[76] - astamps_corrected[51]-9*500)

1091402234432 1091402238849 -83


In [20]:
basis_count = 0
error_count = 0
for (a,b) in zip(ac_indexes,bc_indexes):
    da = achannel[a]
    db = bchannel[b]
    if (da >= 3 and db >= 3):
        #basis HV
        #print(da,db)
        basis_count+=1
        if (da!=db):
            error_count+=1
    if( da<3 and db <3):
        #print (da,db)
        if(da!=db):
            error_count+=1
        basis_count+=1

In [21]:
basis_count

3271412

In [22]:
error_count

123115

In [29]:
QBER = error_count / basis_count
QBER*100

3.7633596746603613

In [54]:
astamps_corrected[0:10]

[1091385445903,
 1091386086360,
 1091386757590,
 1091386908234,
 1091387558548,
 1091387672805,
 1091387746027,
 1091387847428,
 1091387866320,
 1091387966391]

In [55]:
bstamps_corrected[0:10]

[1091385402445,
 1091385441456,
 1091385500418,
 1091385719445,
 1091385757884,
 1091386048629,
 1091386309074,
 1091386410422,
 1091386776858,
 1091387379778]

In [24]:
len (ac_indexes)

3338337

In [26]:
duration = astamps[-1]-astamps[0]
duration / (10**12)

9.966141765334

In [28]:
len(astamps)/10

3434290.3

Singles rate 3.4 Million/s
coincidence rate 333 K/s 
QBER = 3.7 %
coincidence window 500 ps

for coincidence window of 1 ns QBER is around 4.2 %
